# Sami Abdelaim - JC Foster

In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [ ]:
#news = pd.read_excel("/content/news_twitter.xlsx")
#news_handles = list(news['handle'])

#finance = pd.read_excel("/content/finance_twitter.xlsx")
#finance_handles = list(finance['handle'])

#oil = pd.read_excel("/content/oil_twitter.xlsx")
#oil_handles = list(oil['handle'])

#think_tank = pd.read_excel("/content/thinktank_twitter.xlsx")
#think_tank_handles = list(think_tank['handle'])

#world_leaders = pd.read_excel("/content/world_leaders_twitter.xlsx")
#world_leaders_handles = list(world_leaders['handle'])

#gov = pd.read_excel("/content/gov_twitter.xlsx")
#gov_handles = list(gov['handle'])

### Current Category
handles_list = gov_handles

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_urls(handle_list):

    url_list = []

    for handle in handle_list:
      search_url = f"https://api.twitter.com/2/tweets/search/recent?query=-is%3Aretweet%20from%3A{handle}%20lang%3Aen%20(oil%20OR%20gas%20OR%20crude%20OR%20diesel)"
      url_list.append(search_url)
    
    return (url_list)

def query_parameters(start_date, end_date, max_results = 100):

    #change params based on the endpoint you are using
    query_params = {
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [ ]:
#Inputs for the request
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOO4bwEAAAAATby7q8FIA7LX5mAF37Yq0HoSxQo%3DrkBVJZkY1gIyUKbUzFDuUIp5IpN6DV4aoKU8mBkjsZpuhwx7U5'
headers = create_headers(bearer_token)

start_list =    ['2022-05-09T03:00:00.000Z',
                 '2022-05-10T00:00:00.000Z',
                 '2022-05-11T00:00:00.000Z',
                 '2022-05-12T00:00:00.000Z',
                 '2022-05-13T00:00:00.000Z',
                 '2022-05-14T00:00:00.000Z',]

end_list =      ['2022-05-10T00:00:00.000Z',
                 '2022-05-11T00:00:00.000Z',
                 '2022-05-12T00:00:00.000Z',
                 '2022-05-13T00:00:00.000Z',
                 '2022-05-14T00:00:00.000Z',
                 '2022-05-15T00:00:00.000Z']
max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        
        url_list = create_urls(handles_list)
        q_params = query_parameters(start_list[i],end_list[i],max_results)

        for url in range(0,len(url_list)):
          json_response = connect_to_endpoint(url_list[url], headers, q_params, next_token)
          result_count = json_response['meta']['result_count']

          if 'next_token' in json_response['meta']:
              # Save the token to use for next call
              next_token = json_response['meta']['next_token']
              print("Next Token: ", next_token)
              if result_count is not None and result_count > 0 and next_token is not None:
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data.csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(1)                
          # If no next token exists
          else:
              if result_count is not None and result_count > 0:
                  print("-------------------")
                  print("Start Date: ", start_list[i])
                  append_to_csv(json_response, "data.csv")
                  count += result_count
                  total_tweets += result_count
                  print("Total # of Tweets added: ", total_tweets)
                  print("-------------------")
                  time.sleep(1)
              
              #Since this is the final request, turn flag to false to move to the next time period.
              flag = False
              next_token = None
          time.sleep(1)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-09T03:00:00.000Z
# of Tweets added from this response:  1
Total # of Tweets added:  1
-------------------
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-09T03:00:00.000Z
# of Tweets added from this response:  1
Total # of Tweets added:  2
-------------------
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
-------------------
Start Dat

Exception: ignored